In [29]:
# Helpers
# from collections import Counter, defaultdict
# import numpy as np
# from itertools import chain
import pickle
import json

# #DB
# from django.db.models import Case, IntegerField, Value

# # Plot
# import matplotlib.pyplot as plt
# import seaborn as sns

In [2]:
folder = "data/recommendation/"

### Creating datasets

Two different datasets will be created:
- Students from the random/EER experiment with 54 different problems
- Students throughout the semester with 65 different problems 

#### Dataset 1

In [3]:
# Filter used problems and solutions
last_id = 132
problems = Problem.objects.filter(id__lte=last_id)
solutions = Solution.objects.filter(problem__in=problems, ignore=False).values_list('problem', flat=True)
problems = solutions.order_by('problem').values_list(
    'problem', flat=True).distinct('problem')
print("Total number of questions: %d" % problems.count())

Total number of questions: 48


In [4]:
# Get all students
user_blacklist = UserProfile.objects.filter(professor__user__username='sem_professor')

data = UserLog.objects.filter(problem__in=problems).exclude(
    user__userprofile__in=user_blacklist).order_by(
    "timestamp").values("user__id", "problem_id", "outcome", "solution", "timestamp")

# Save data
with open("%s/dataset_1.pkl" % (folder), "wb") as pklfile:
    pickle.dump(list(data), pklfile)
data.count()

4401

#### Dataset 2

In [5]:
problems = Problem.objects.exclude(chapter=None)
solutions = Solution.objects.filter(problem__in=problems, ignore=False).values_list('problem', flat=True)
problems = solutions.order_by('problem').values_list(
    'problem', flat=True).distinct('problem')
print("Total number of questions: %d" % problems.count())

Total number of questions: 65


In [6]:
# Get all students
user_blacklist = UserProfile.objects.filter(professor__user__username='sem_professor')

data = UserLog.objects.filter(problem__in=problems).exclude(
    user__userprofile__in=user_blacklist).order_by(
    "timestamp").values("user__id", "problem_id", "outcome", "solution", "timestamp")

# Save data
with open("%s/dataset_2.pkl" % (folder), "wb") as pklfile:
    pickle.dump(list(data), pklfile)
data.count()

29273

In [17]:
list(data)

[{'user__id': 393,
  'problem_id': 727,
  'outcome': 'P',
  'solution': '# Coloque um comentário dizendo o que a função faz\n# Escolha nomes elucidativos para suas variáveis\n# int -> bool\nfrom math import *\ndef absoluto(num):\n    return abs(num)',
  'timestamp': datetime.datetime(2019, 8, 21, 17, 4, 49, 829685, tzinfo=<UTC>)},
 {'user__id': 393,
  'problem_id': 727,
  'outcome': 'P',
  'solution': '# Coloque um comentário dizendo o que a função faz\n# Escolha nomes elucidativos para suas variáveis\n# int -> bool\nfrom math import *\ndef absoluto(num):\n    return abs(num)',
  'timestamp': datetime.datetime(2019, 8, 21, 17, 4, 50, 5231, tzinfo=<UTC>)},
 {'user__id': 422,
  'problem_id': 727,
  'outcome': 'S',
  'solution': '# Coloque um comentário dizendo o que a função faz\n# Escolha nomes elucidativos para suas variáveis\n# int -> bool\ndef absoluto(num):',
  'timestamp': datetime.datetime(2019, 8, 22, 13, 47, 33, 879029, tzinfo=<UTC>)},
 {'user__id': 424,
  'problem_id': 727,
  '

#### Test cases

In [41]:
solutions = Solution.objects.filter(problem__in=problems).values("problem_id", "content", "header",
                                                                 "problem__testcase__content"
                                                                ).order_by("content")

data = []
old_content = ""
for item in solutions:
    if item["content"] == old_content:
        data[-1]["test_case"].append(json.loads(item["problem__testcase__content"]))
    else:
        item_data = {
            "problem_id": item["problem_id"],
            "solution": item["content"],
            "function_call": item["header"],
            "test_case": [json.loads(item["problem__testcase__content"])]
        }
        data.append(item_data)
    old_content = item["content"]
    
# Save data
with open("%s/test_cases.pkl" % (folder), "wb") as pklfile:
    pickle.dump(list(data), pklfile)

In [46]:
data[0]

{'problem_id': 727,
 'solution': 'def absoluto(num):\r\n    if num >= 0:\r\n        return num\r\n    else:\r\n        return -num',
 'function_call': 'absoluto',
 'test_case': [[-31], [19], [7], [-16], [0]]}

In [45]:
for item in data:
    for tc in item["test_case"]:
        print(tc)
        # Transform solution into python function
        function_obj = compile(item["solution"], 'solution', 'exec')
        exec(function_obj)
        answer = eval(item["function_call"])(*tc)

[-31]
31
[19]
19
[7]
7
[-16]
16
[0]
0
[16, 60, 7]
Insuficiente
[8, 80, 1]
Suficiente
[12, 26, 3]
Insuficiente
[10, 46, 5]
Insuficiente
[17, 74, 6]
Insuficiente
[7, 83, 3]
Suficiente
[7, 27, 9]
Insuficiente
[8, 82, 3]
Suficiente
[15, 40, 1]
Suficiente
[8, 72, 6]
Suficiente
[4, 61, 7]
Suficiente
[14, 71, 5]
Suficiente
[19, 15, 5]
Insuficiente
[6, 84, 7]
Suficiente
[3, 37, 4]
Suficiente
[2, 21, 4]
Suficiente
[2, 13, 1]
Suficiente
[17, 36, 7]
Insuficiente
[8, 37, 9]
Insuficiente
[4, 54, 3]
Suficiente
[13, 34, 9]
Insuficiente
[3, 41, 6]
Suficiente
[16, 58, 8]
Insuficiente
[14, 88, 4]
Suficiente
[10, 4, 9]
Insuficiente
[10, 75, 8]
Insuficiente
[1, 97, 5]
Suficiente
[9, 7, 2]
Insuficiente
[10, 44, 3]
Suficiente
[4, 58, 7]
Suficiente
[14, 26, 3]
Insuficiente
[18, 38, 5]
Insuficiente
[19, 41, 5]
Insuficiente
[19, 62, 3]
Suficiente
[5, 40, 3]
Suficiente
[5, 39, 8]
Insuficiente
[14, 44, 8]
Insuficiente
[14, 74, 2]
Suficiente
[5, 30, 2]
Suficiente
[9, 8, 5]
Insuficiente
[10, 94, 6]
Suficiente
[15,

XXIV
[613]
DCXIII
[646]
DCXLVI
[547]
DXLVII
[833]
DCCCXXXIII
[886]
DCCCLXXXVI
[410]
CDX
[[7, 0, 6], [5, 1, 4]]
[7, 5, 0, 1, 6, 4]
[[8, 6, 4], [0, 5, 2]]
[8, 0, 6, 5, 4, 2]
[[5, 8, 2], [3, 6, 1]]
[5, 3, 8, 6, 2, 1]
[[2, 5, 4], [4, 5, 9]]
[2, 4, 5, 5, 4, 9]
[[5, 6, 9], [0, 2, 8]]
[5, 0, 6, 2, 9, 8]
[[1, 3, 2], [3, 1, 2]]
[1, 3, 3, 1, 2, 2]
[[1, 9, 6], [1, 6, 2]]
[1, 1, 9, 6, 6, 2]
[[8, 4, 9], [6, 4, 7]]
[8, 6, 4, 4, 9, 7]
[[3, 9, 4], [9, 5, 3]]
[3, 9, 9, 5, 4, 3]
[[0, 5, 6], [1, 2, 8]]
[0, 1, 5, 2, 6, 8]
['é o que contarei no outro capítulo']
capítulo outro no contarei que o é
['tu quoque brute?']
brute? quoque tu
['nascera muito depois daquelas festas célebres']
célebres festas daquelas depois muito nascera
['anda apanhar um capotinho capitu dizia-lhe ele']
ele dizia-lhe capitu capotinho um apanhar anda
['conte-me as festas da coroação']
coroação da festas as conte-me
['tio cosme ensinou-lhe gamão']
gamão ensinou-lhe cosme tio
['e quanto valia cada sestércio?']
sestércio? cada valia qua

razvo
['equitativo']
ivoequitat
['ogivado']
adoogiv
['cabular']
larcabu
['troçar']
çartro
['festinhas']
hasfestin
['crápula']
ulacráp
['escorrer']
rerescor
['delínquo']
quodelín
['subtractivo']
ivosubtract
['alcatraz']
razalcat
['generante']
ntegenera
['lixoso']
osolix
['trajo']
ajotr
['esticanço']
nçoestica
['previne']
ineprev
['inclusão']
sãoinclu
['centro']
trocen
['cólico']
icocól
['espaldar']
darespal
['arborista']
staarbori
['borbulhento']
ntoborbulhe
['higienizar']
zarhigieni
['adviéreis']
eisadviér
['candelabro']
brocandela
['boateiro']
iroboate
['exultar']
tarexul
['cacheiro']
irocache
['traziam']
iamtraz
['penúltimo']
imopenúlt
['particularista']
staparticulari
['Schiller']
lerSchil
['intercelular']
larintercelu
['humildade']
adehumild
['delirar']
rardeli
['sigilar']
larsigi
['ressentir']
tirressen
['advirdes']
desadvir
['convêm']
vêmcon
['linguístico']
icolinguíst
['persecução']
çãopersecu
['destingir']
girdestin
['intravável']
velintravá
['entretendo']
ndoentrete
['barão']


['ressentir', 'f', 7]
ressentfr
['sorrir', 's', 4]
sorrsr
['candelabro', 'v', 0]
vandelabro
['crestar', 'q', 0]
qrestar
['vagão', 'e', 4]
vagãe
['gauchai', 'f', 4]
gaucfai
['fanático', 'r', 2]
farático
['lixoso', 'n', 0]
nixoso
['troçar', 'm', 4]
troçmr
['vivificativo', 's', 4]
vivisicativo
['virginalizar', 'i', 11]
virginalizai
['escorrer', 'w', 1]
ewcorrer
['famigerado', 'd', 0]
damigerado
['terreno', 'l', 1]
tlrreno
['europeízem', 'p', 0]
puropeízem
['arborista', 'q', 3]
arbqrista
['isotónico', 'k', 7]
isotóniko
['multimédia', 'y', 6]
multimydia
['borbulhento', 'f', 10]
borbulhentf
['saberdes', 'w', 0]
waberdes
['filonianos', 'o', 0]
oilonianos
['voraz', 'm', 2]
vomaz
['subtractivo', 'j', 10]
subtractivj
['amoladura', 'h', 6]
amoladhra
['sagaz', 'p', 2]
sapaz
['centro', 'j', 5]
centrj
['intravável', 't', 4]
intrtvável
['persecução', 'b', 4]
persbcução
['Schiller', 'y', 5]
Schilyer
['higienizar', 'm', 1]
hmgienizar
['OGMA', 'l', 0]
lGMA
['séquito', 'm', 2]
sémuito
['penúltimo', 'g', 